In [ ]:
%reload_ext autoreload
%autoreload 2

from src.acnets.deep import LEMONDataModule, Julia2018DataModule
import numpy as np
import torch
from torch import nn
import torch.nn.functional  as F
import pywt
import matplotlib.pyplot as plt
import seaborn as sns

from src.acnets.deep.cvae import CVAE

In [ ]:
datamodule = LEMONDataModule(n_subjects=8, test_ratio=None)
# datamodule = Julia2018DataModule()
datamodule.setup()
batch = datamodule.train[:]

In [ ]:
datamodule = LEMONDataModule(n_subjects=8, test_ratio=None)
datamodule.setup()

batch = datamodule.train[:]
x_wavelets = batch[5] # -> shape: (subjects, wavelets, regions)

n_channels = x_wavelets.shape[2]
n_wavelets = 32
n_embeddings = 16

x = x_wavelets[:, :n_wavelets, :].transpose(1, 2)  # -> shape: (subjects, regions, wavelets)

h, x_recon, loss = CVAE(n_channels, n_embeddings)(x)
x.shape, h.shape, x_recon.shape, loss


In [ ]:
%reload_ext autoreload
%autoreload 2

from src.acnets.deep import MultiHeadWaveletModel, LEMONDataModule, Julia2018DataModule

lemon_datamodule = LEMONDataModule(
    atlas='aal', kind='partial correlation',
    test_ratio=.5, val_ratio=.5,
    n_subjects=16, batch_size=201)
lemon_datamodule.setup()

julia2018_datamodule = Julia2018DataModule(
    atlas='aal', kind='partial correlation',
    test_ratio=.5, batch_size=16)
julia2018_datamodule.setup()

n_regions = lemon_datamodule.train[0][0].shape[1]
n_wavelets = 124
n_embeddings = 8
segment_length = 32

model = MultiHeadWaveletModel(n_regions, n_wavelets, n_embeddings, segment_length)

trainer = model.fit(lemon_datamodule, max_epochs=300, with_classifier=False)
tuner = model.fit(julia2018_datamodule, max_epochs=1000, with_classifier=True)

In [ ]:
# Visualize embedding space

from umap import UMAP
from hdbscan import HDBSCAN
import seaborn as sns
import pandas as pd

x_ft = torch.concat([
    julia2018_datamodule.train[:][5],
    julia2018_datamodule.test[:][5]
], dim=0)

y_ft = torch.concat([
    julia2018_datamodule.train[:][6],
    julia2018_datamodule.test[:][6]
], dim=0)

x_pt = lemon_datamodule.train[:][0][:, :n_wavelets, :]

_, h_ft,_ = model.forward(x_ft.cuda())
_, h_pt,_ = model.forward(x_pt.cuda())

x_ft.shape, h_ft.shape

# umap_model = UMAP(n_components=2, n_neighbors=2).fit(x_pt.reshape(x_pt.shape[0], -1))
# x_ft_reduced = umap_model.transform(x_ft.reshape(x_ft.shape[0], -1))
# x_pt_reduced = umap_model.transform(x_pt.reshape(x_pt.shape[0], -1))
# sns.scatterplot(x=x_ft_reduced[:, 0], y=x_ft_reduced[:, 1], hue=y_ft)
# sns.scatterplot(x=x_pt_reduced[:, 0], y=x_pt_reduced[:, 1], color='gray', alpha=.2)